# Slope Prediction for Feed-Forward Process Control (without time)

In [14]:
import pandas as pd
import numpy as np
import os
import sys
import sklearn.model_selection
import matplotlib.pyplot as plt
import warnings
import sklearn, sklearn.preprocessing, sklearn.pipeline, sklearn.model_selection
import sklearn.svm, sklearn.tree, sklearn.ensemble, sklearn.neural_network
import sklearn.multioutput
import numpy
import functools, operator, itertools
import time
import scipy.stats, scipy
# import scikitlearn_plus.neural_network
warnings.filterwarnings('ignore')

## Get Starting Data
This data was generated from the data processing notebook

In [15]:
home_dir = os.getcwd().split('/notebooks')[0]
sys.path.append(home_dir)
home_dir
smooth_data = pd.read_csv(f'{home_dir}/processed_data/smooth_data.csv')
smooth_data.set_index(['composition','trial','time'], drop=True, inplace=True)
print(f'Shape of the smooth data: {smooth_data.shape[0]} rows by {smooth_data.shape[1]} columns')

Shape of the smooth data: 856 rows by 24 columns


## Create train/test sets (Changed from original train/test
New: <br>
Training data comes from compostions 1-5, 9, 10.<br>
Test data comes from compostions 6, 7, and 8.
<br>
<br>
Old: <br>
Training data comes from compostions 1-7.<br>
Test data comes from compostions 8, 9, and 10.

In [16]:
train_validation_data = smooth_data.loc[[1,2,3,4,5,9,10]]
print(f'Shape of the train_validation data: {train_validation_data.shape[0]} rows by {train_validation_data.shape[1]} columns')
test_data = smooth_data.loc[[6,7,9]]
print(f'Shape of the test data: {test_data.shape[0]} rows by {test_data.shape[1]} columns')

Shape of the train_validation data: 550 rows by 24 columns
Shape of the test data: 306 rows by 24 columns


## Define a function that generates the X array and y array for ML model training

The parameter 'input_data' is used to specify whether raw data or the polynomial smoothed data will be used to train the model <br>
The parameter 'conditions_to_include' is a list of the conditions to include in the returned arrays

In [17]:
def get_X_y_arrays(data):

    data_copy = data.copy()
    data_copy.reset_index(inplace=True)
    X = data_copy [['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol', 'CO', 'CO2', 'H2', 'flow rate']]
    y = data_copy [['acetate_Δ', 'biomass_Δ', 'butanol_Δ', 'butyrate_Δ', 'ethanol_Δ']]
 
    return np.array(X), np.array(y)

In [18]:
X_train_validation, y_train_validation = get_X_y_arrays(train_validation_data)
X_test, y_test = get_X_y_arrays(test_data)

print(f'Shape of the train_validation X array: {X_train_validation.shape[0]} rows by {X_train_validation.shape[1]} columns')
print(f'Shape of the train_validation y array: {y_train_validation.shape[0]} rows by {y_train_validation.shape[1]} columns')
print(f'Shape of the test X array: {X_test.shape[0]} rows by {X_test.shape[1]} columns')
print(f'Shape of the test y array: {y_test.shape[0]} rows by {y_test.shape[1]} columns')

Shape of the train_validation X array: 550 rows by 9 columns
Shape of the train_validation y array: 550 rows by 5 columns
Shape of the test X array: 306 rows by 9 columns
Shape of the test y array: 306 rows by 5 columns


## Train 25 different models using 5 algorithms for each of the 5 outputs
algorithms = gradient boosting, random forest, support vector, neural net, lasso <br>
outputs = acetate, biomass, butanol, butyrate, ethanol

In [6]:
def gen_NN_fixed_n_layers(n_layers, n_neurons, neuron_step):
    """Generate NN hidden_layer_sizes of n_layers and up to n_neurons per layer 
    """
    # print (n_layers)
    if n_layers == 1: 
        return [[i] for i in range(neuron_step, n_neurons+1, neuron_step)]
    else:
        pairs =  [  (i,  tail) for tail in gen_NN_fixed_n_layers(n_layers-1, n_neurons+1, neuron_step) for i in range(neuron_step, n_neurons+1, neuron_step) ]
        return [[i]+ t for (i, t) in pairs]

# print (gen_NN_fixed_n_layers(4, 10, 5))

def gen_NN_uni(n_layers, n_neurons, layer_step, neuron_step):
    """Generate hidden layers of various number of layers and number of neurons 
    """ 
    various_NNs = [ gen_NN_fixed_n_layers(i , n_neurons, neuron_step) for i in range(2, n_layers+1, layer_step)]
    return  functools.reduce(operator.add, various_NNs)

def model_selection(X, y):
    trained_models = {}
    model_cfgs = {
        "nn":{
            'estimator': sklearn.neural_network.MLPRegressor(shuffle=True),
#            Test grid
            'param_grid':   {
                # hidden_layer_sizes made the search space many order of magnitudes larger
                'activation':         ['tanh', 'logistic', 'relu'], 
                'max_iter':           [400*i for i in range(1, 2)]
            }
#             Full grid
#             'param_grid':   {
#                 'hidden_layer_sizes': gen_NN_uni(5, 100, 1, 10),  
#                 'activation':         ['tanh', 'logistic', 'relu'], 
#                 'max_iter':           [400*i for i in range(1, 10, 2)]
#             }                
        },
        "svm_rbf":{
                'estimator': sklearn.svm.SVR(kernel='rbf'),
#               Test grid
                'param_grid':   {
                    'C':       [10**i for i in range(-1, 1)], 
                    'epsilon': [10**i for i in range(-1, 1)],
                }
#                  Full grid
#                 'param_grid':   {
#                     'C':       [10**i for i in range(-5, 5)], 
#                     'epsilon': [10**i for i in range(-5, 5)],
# #                     'gamma':   [10**i for i in range(-5, 5)] # gamma gave me an error
#                 }
        },

        "rf":{
            'estimator': sklearn.ensemble.RandomForestRegressor(),
#            Test grid
            'param_grid':   {
                'n_estimators': [10*i for i in range(1, 2)],
                'max_depth':     [2*i for i in range(1, 1+1)],
            }
#             Full grid 
#             'param_grid':   {
#                 'n_estimators': [10*i for i in range(1, 20)],
#                 'max_depth':     [2*i for i in range(20)], 
# #                'max_samples': [0.05*i for i in range(1, 10+1)] # max samples gave me an error
#             }
        }
    }
    
    # scale data once 
    Scaler = sklearn.preprocessing.MinMaxScaler()
    X = Scaler.fit_transform(X, y)
        
    trained_model_dictionary = {}
        
    for index, output in enumerate(['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']):
        print(f'{output}\n')
        for model_name, model_conf in model_cfgs.items():
            print (model_name)
            search = sklearn.model_selection.GridSearchCV(
                estimator = model_conf["estimator"], 
                param_grid = model_conf["param_grid"], 
#                 scoring = ["neg_mean_absolute_percentage_error","r2"],
#                 neg_mean_absolute_percentage_error is not a valid, but neg_mean_absolute_error is
                scoring = "r2",
                refit = True,
                cv = sklearn.model_selection.ShuffleSplit(n_splits=10, test_size=0.1, random_state=0), 
                n_jobs=30,
                verbose=3
            )
        
            # This line uses only the column related to the output rather than the whole output matrix
            # Need to assign the slice to a new variable, so that data for other outputs is not lost
            y_output=y[:,index]

            search.fit(X, y_output)
            print("Best CV score: %0.3f:" % search.best_score_)
            print("Best parameters:",  search.best_params_, '\n')
            trained_models[model_name] = search 

        trained_model_dictionary[output] = trained_models
        
    return trained_model_dictionary

trained_model_dictionary = model_selection(X_train_validation, y_train_validation)

acetate

nn
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  30 | elapsed:    8.8s remaining:   57.0s
[Parallel(n_jobs=30)]: Done  15 out of  30 | elapsed:   10.6s remaining:   10.6s
[Parallel(n_jobs=30)]: Done  26 out of  30 | elapsed:   12.3s remaining:    1.9s
[Parallel(n_jobs=30)]: Done  30 out of  30 | elapsed:   12.5s finished


Best CV score: 0.557:
Best parameters: {'activation': 'relu', 'max_iter': 400} 

svm_rbf
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   9 out of  40 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    0.3s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=30)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.


Best CV score: 0.164:
Best parameters: {'C': 1, 'epsilon': 0.1} 

rf
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best CV score: 0.699:
Best parameters: {'max_depth': 2, 'n_estimators': 10} 

biomass

nn
Fitting 10 folds for each of 3 candidates, totalling 30 fits


...

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   9 out of  40 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=30)]: Done  23 out of  40 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=30)]: Done  37 out of  40 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    0.1s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=30)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  30 | elapsed:    6.0s remaining:   39.2s
[Parallel(n_jobs=30)]: Done  15 out of  30 | elapsed:   10.2s remaining:   10.2s
[Parallel(n_jobs=30)]: Done  26 out of  30 | elapsed:   11.0

Best CV score: 0.856:
Best parameters: {'activation': 'relu', 'max_iter': 400} 

svm_rbf
Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best CV score: 0.243:
Best parameters: {'C': 1, 'epsilon': 1} 

rf
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best CV score: 0.435:
Best parameters: {'max_depth': 2, 'n_estimators': 10} 

ethanol

nn
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   9 out of  40 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=30)]: Done  23 out of  40 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  37 out of  40 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    0.1s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   4 out of  30 | elapsed:    7.6s remaining:   49.7s
[Parallel(n_jobs=30)]: Done  15 out of  30 | elapsed:    9.6s remaining:    9.6s
[Parallel(n_jobs=30)]: Done  26 out of  30 | elapsed:   10.7

Best CV score: 0.569:
Best parameters: {'activation': 'relu', 'max_iter': 400} 

svm_rbf
Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best CV score: 0.379:
Best parameters: {'C': 1, 'epsilon': 1} 

rf
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best CV score: 0.633:
Best parameters: {'max_depth': 2, 'n_estimators': 10} 



[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   9 out of  40 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=30)]: Done  23 out of  40 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  37 out of  40 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  40 out of  40 | elapsed:    0.1s finished
[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:    0.0s finished


In [11]:
optimized_models = {
    "acetate": {
        "nn": sklearn.neural_network.MLPRegressor(
            shuffle=True, 
            activation = 'tanh', 
            hidden_layer_sizes = [20,100],
            max_iter = 5000
        ),
        "svm_rbf": sklearn.svm.SVR(
            kernel = 'rbf', 
            C = 10000, 
            epsilon = 0.1, 
            gamma = 10
        ),
        'rf': sklearn.ensemble.RandomForestRegressor(
            max_depth = 30,
#             max_samples = 0.5,
            n_estimators = 60
        )
    },
    "biomass": {
        "nn": sklearn.neural_network.MLPRegressor(
            shuffle=True, 
            activation = 'relu', 
            hidden_layer_sizes = [100, 100, 80],
            max_iter = 5000
        ),
        "svm_rbf": sklearn.svm.SVR(
            kernel = 'rbf', 
            C = 10, 
            epsilon = 0.0001, 
            gamma = 10
        ),
        'rf': sklearn.ensemble.RandomForestRegressor(
            max_depth = 20,
#             max_samples = 0.5,
            n_estimators = 60
        )
    },
    "butanol": {
        "nn": sklearn.neural_network.MLPRegressor(
            shuffle=True, 
            activation = 'tanh', 
            hidden_layer_sizes = [20, 100, 60, 20],
            max_iter = 5000
        ),
        "svm_rbf": sklearn.svm.SVR(
            kernel = 'rbf', 
            C = 10000, 
            epsilon = 0.01, 
            gamma = 10
        ),
        'rf': sklearn.ensemble.RandomForestRegressor(
            max_depth = 12,
#             max_samples = 0.5,
            n_estimators = 40
        )
    },
    "butyrate": {
        "nn": sklearn.neural_network.MLPRegressor(
            shuffle=True, 
            activation = 'tanh', 
            hidden_layer_sizes = [60, 60, 100, 20],
            max_iter = 5000
        ),
        "svm_rbf": sklearn.svm.SVR(
            kernel = 'rbf', 
            C = 1000, 
            epsilon = 0.01, 
            gamma = 10
        ),
        'rf': sklearn.ensemble.RandomForestRegressor(
            max_depth = 34,
#             max_samples = 0.5,
            n_estimators = 80
        )
    },
    "ethanol": {
        "nn": sklearn.neural_network.MLPRegressor(
            shuffle=True, 
            activation = 'tanh', 
            hidden_layer_sizes = [60, 100],
            max_iter = 5000
        ),
        "svm_rbf": sklearn.svm.SVR(
            kernel = 'rbf', 
            C = 10000, 
            epsilon = 0.1, 
            gamma = 10
        ),
        'rf': sklearn.ensemble.RandomForestRegressor(
            max_depth = 38,
#             max_samples = 0.5,
            n_estimators = 170
        )
    },
}
    


In [13]:
for output in ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']:
    for model in ['nn', 'svm_rbf', 'rf']:
        
        print(optimized_models[output][model])
        print()
    print()

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=[20, 100], learning_rate='constant',
             learning_rate_init=0.001, max_iter=5000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

SVR(C=10000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=10,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=60,
                   

In [ ]:
Scores = [] # to store best scores from each type of models, the metric is the same as those used in training
Predictions = [] # to store predictions from each type of models. 

for model_name, model in Trained_models.items():
    print ("using model ", model_name)
    prediction = model.predict(X_test) # 2-D numpy array, each row is a sample, each column is one output
    output_name = ['acetate_Δ', 'biomass_Δ', 'butanol_Δ', 'butyrate_Δ', 'ethanol_Δ']
    for i in range(5):
        (r, p) = scipy.stats.pearsonr(prediction[:,i], y_test[:,i])
        print (output_name[i], ", pearson's CC :", r)
        # (r, p) = scipy.stats.spearmanr(prediction[:,i], y_test[:,i])
        # print (output_name[i], ", spearman's' CC :", r)

    

# Below are from Garrett's original result 

## Plot evaluation metrics
Plot r$^2$ values first

In [ ]:
for output in ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']:
    plot_evaluation_metrics(output, 'r2')

Plot normalized mean squared error

In [ ]:
# for output in ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']:
#     plot_evaluation_metrics(output, 'norm_rmse')

## Plot scatterplots of slope fits
Plot alcohols

In [ ]:
outputs = ['ethanol', 'butanol']
models = ['gradient boosting', 'random forest', 'support vector', 'neural net', 'lasso']

fig, ax = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle(f'Slope Predictions (without time)')

for x in range(2):
    for y in range(5):
        ax[x, y].title.set_text(f'{outputs[x]} {models[y]}')
        
        ax[x, y].scatter(train_data[outputs[x]], train_predictions[models[y]][outputs[x]])
        ax[x, y].scatter(validation_data[outputs[x]], validation_predictions[models[y]][outputs[x]])
        ax[x, y].scatter(test_data[outputs[x]], test_predictions[models[y]][outputs[x]])
        
        minimum = min(pd.concat([
            test_data[outputs[x]],
            validation_data[outputs[x]],
            train_data[outputs[x]], 
            test_predictions[models[y]][outputs[x]],
            validation_predictions[models[y]][outputs[x]],
            train_predictions[models[y]][outputs[x]]
        ], axis=0))

        maximum = max(pd.concat([
            test_data[outputs[x]],
            validation_data[outputs[x]],
            train_data[outputs[x]],
            test_predictions[models[y]][outputs[x]],
            validation_predictions[models[y]][outputs[x]],
            train_predictions[models[y]][outputs[x]]
        ], axis=0))

        ax[x, y].plot([minimum, maximum], [minimum, maximum], 'r') #row=0, col=0

plt.savefig(f'{home_dir}/figures/slope_figures_with_time/slope_scatterplots_without_time.png', dpi=100)
plt.show()

Plot others

In [ ]:
outputs = ['acetate', 'butyrate', 'biomass']
models = ['gradient boosting', 'random forest', 'support vector', 'neural net', 'lasso']

fig, ax = plt.subplots(3, 5, figsize=(20, 12))
fig.suptitle(f'Slope Predictions (without time)')

for x in range(3):
    for y in range(5):
        ax[x, y].title.set_text(f'{outputs[x]} {models[y]}')
        
        ax[x, y].scatter(train_data[outputs[x]], train_predictions[models[y]][outputs[x]])
        ax[x, y].scatter(validation_data[outputs[x]], validation_predictions[models[y]][outputs[x]])
        ax[x, y].scatter(test_data[outputs[x]], test_predictions[models[y]][outputs[x]])
        
        minimum = min(pd.concat([
            test_data[outputs[x]],
            validation_data[outputs[x]],
            train_data[outputs[x]], 
            test_predictions[models[y]][outputs[x]],
            validation_predictions[models[y]][outputs[x]],
            train_predictions[models[y]][outputs[x]]
        ], axis=0))

        maximum = max(pd.concat([
            test_data[outputs[x]],
            validation_data[outputs[x]],
            train_data[outputs[x]],
            test_predictions[models[y]][outputs[x]],
            validation_predictions[models[y]][outputs[x]],
            train_predictions[models[y]][outputs[x]]
        ], axis=0))

        ax[x, y].plot([minimum, maximum], [minimum, maximum], 'r') #row=0, col=0

plt.savefig(f'{home_dir}/figures/slope_figures_without_time/slope_scatterplots_without_time_all.png', dpi=100)
plt.show()

## Feature Importance
Define a function to get feature importances

In [ ]:
def get_feature_importances(model):
    outputs = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    features = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol', 'CO', 'CO2', 'H2', 'flow rate']

    array_list = []

    for i in range(5):
        feature_importance_array = model.estimators_[i].steps[1][1].best_estimator_.feature_importances_
        array_list.append(list(feature_importance_array))
    df = pd.DataFrame(array_list, columns = features, index = outputs)  
    return df

Get feature importance values

In [ ]:
model_list = [
    trained_models['gradient boosting'],
    trained_models['random forest'],
    ]

for model in model_list:
    display(get_feature_importances(model))

Define a function to plot feature importance

In [ ]:
def plot_feature_importance_metabolites(model_name):
    model = trained_models[model_name]
    data = get_feature_importances(model)
    
    ethanol_data = data.iloc[4]
    butanol_data = data.iloc[2]
    
    ethanol_data = ethanol_data[:5]
    butanol_data = butanol_data[:5]
    
    labels = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol'] #, 'CO', 'CO2', 'H2', 'flow rate']
    
    x = np.arange(len(labels))  # the label locations
    width = 0.25  # the width of the bars

    fig, ax = plt.subplots(figsize=(8,5))
    rects5 = ax.bar(x - 0.5*width, ethanol_data , width, label='Ethanol Rate')
    rects3 = ax.bar(x + 0.5*width , butanol_data , width, label='Butanol Rate')
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(f'Importance')
    ax.set_yscale('log')
    ax.set_title(f'Feature Importance for {model_name} Slope Predictions (without time)')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()
    plt.savefig(f'{home_dir}/figures/slope_figures_without_time/slope_feature_importance_{model_name}_metabolites_without_time.png', dpi=100)
    plt.show()

In [ ]:
plot_feature_importance_metabolites('gradient boosting')

Plot feature importance gradient boosting and random forest

In [ ]:
plot_feature_importance_metabolites('random forest')

In [ ]:
def plot_feature_importance_gases(model_name):
    model = trained_models[model_name]
    data = get_feature_importances(model)
    
    ethanol_data = data.iloc[4]
    butanol_data = data.iloc[2]
    
    ethanol_data = ethanol_data[5:]
    butanol_data = butanol_data[5:]
    
    labels = ['CO', 'CO2', 'H2', 'flow rate']
    
    x = np.arange(len(labels))  # the label locations
    width = 0.25  # the width of the bars

    fig, ax = plt.subplots(figsize=(8,5))
    rects5 = ax.bar(x - 0.5*width, ethanol_data , width, label='Ethanol Rate')
    rects3 = ax.bar(x + 0.5*width , butanol_data , width, label='Butanol Rate')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(f'Importance')
    ax.set_yscale('log')
    ax.set_title(f'Feature Importance for {model_name} Slope Predictions (without time)')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    
    fig.tight_layout()
    plt.savefig(f'{home_dir}/figures/slope_figures_without_time/slope_feature_importance_gases_{model_name}.png', dpi=100)
    plt.show()

In [ ]:
plot_feature_importance_gases('gradient boosting')

In [ ]:
plot_feature_importance_gases('random forest')